In [1]:
import pandas as pd

In [2]:
url = "https://raw.githubusercontent.com/deysigalvez38/Archivos/main/GSAF5_clean.csv"
df = pd.read_csv(url)

In [3]:
# paso 1:  Observar el dataframe
df.head()
df.columns

Index(['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Fatal Y/N', 'Time', 'Species ',
       'Source', 'pdf', 'href formula', 'href', 'Case Number', 'Case Number.1',
       'original order', 'Unnamed: 21'],
      dtype='object')

In [12]:
df.rename(columns={'Fatal Y/N': 'Fatality'}, inplace=True)
df.columns


Index(['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Fatality', 'Time', 'Species ',
       'Source', 'pdf', 'href formula', 'href', 'Case Number', 'Case Number.1',
       'original order', 'Unnamed: 21'],
      dtype='object')

In [13]:
df['Fatality'].unique()


array(['N', 'Not stated', 'Y', ..., 'ND-0004-Ahmun.pdf',
       'F. Schwartz, p.23; C. Creswell, GSAF',
       'http://sharkattackfile.net/spreadsheets/pdf_directory/ND-0002-JulesPatterson.pdf'],
      shape=(2639,), dtype=object)

In [15]:
# Elimina espacios al inicio/final y reemplaza caracteres raros
df.columns = df.columns.str.strip()
print(df.columns.tolist())



['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatality', 'Time', 'Species', 'Source', 'pdf', 'href formula', 'href', 'Case Number', 'Case Number.1', 'original order', 'Unnamed: 21']


In [16]:
# Buscar columna que contenga la palabra 'Fatal'
fatal_col = [col for col in df.columns if 'Fatal' in col]
print(fatal_col)


['Fatality']


In [19]:
col_name = fatal_col[0]  # usa el nombre detectado automáticamente

df['Fatality_clean'] = df[col_name].map({'Y': 'Yes', 'N': 'No', 'Not stated': 'Unknown'})
df['Fatality_clean'] = df['Fatality_clean'].fillna('Unknown')


print(df['Fatality_clean'].unique())


['No' 'Unknown' 'Yes']


In [25]:
df['Activity'].unique()

array(['Fishing/swimming', 'Surfing', 'Swimming', ..., '20 Fijians',
       'A chiefessF', 'Ahmun'], shape=(2101,), dtype=object)

In [26]:
# Quitar espacios al inicio/final y pasar a formato título
df['Activity_clean'] = df['Activity'].str.strip().str.title()


In [27]:
valid_activities = [
    'Swimming', 'Surfing', 'Spearfishing', 'Bathing', 'Snorkeling',
    'Bodyboarding', 'Diving', 'Wading', 'Standing', 'Fishing'
]


In [28]:
df['Activity_clean'] = df['Activity_clean'].apply(lambda x: x if x in valid_activities else 'Other')


In [29]:
df['Activity_clean'].value_counts()


Activity_clean
Other           3492
Surfing         1125
Swimming         963
Fishing          445
Spearfishing     354
Wading           168
Bathing          152
Snorkeling       119
Diving           117
Standing         111
Bodyboarding       1
Name: count, dtype: int64

In [30]:
# Top actividades con ataques fatales
df[df['Fatality_clean']=='Yes']['Activity_clean'].value_counts().head(10)


Activity_clean
Other           241
Swimming        196
Surfing          56
Spearfishing     38
Bathing          21
Snorkeling       15
Fishing          11
Diving           11
Wading           11
Standing         11
Name: count, dtype: int64

In [31]:
df.groupby('Activity_clean')['Fatality_clean'].value_counts(normalize=True).unstack().fillna(0)


Fatality_clean,No,Unknown,Yes
Activity_clean,,,
Bathing,0.164474,0.697368,0.138158
Bodyboarding,1.000000,0.000000,0.000000
Diving,0.299145,0.606838,0.094017
Fishing,0.303371,0.671910,0.024719
Other,0.258305,0.672680,0.069015
Snorkeling,0.655462,0.218487,0.126050
Spearfishing,0.548023,0.344633,0.107345
Standing,0.693694,0.207207,0.099099
Surfing,0.720000,0.230222,0.049778


In [6]:
# =======================================================
# IMPORTS
# =======================================================
import pandas as pd
import unicodedata
import re
# =======================================================
# CARGA
# =======================================================
url = "https://raw.githubusercontent.com/deysigalvez38/Archivos/main/GSAF5_clean.csv"
df = pd.read_csv(url)
# =======================================================
# COLUMNAS CLAVE (incluimos fatalidad si existe)
# =======================================================
# columnas base que queremos siempre
mantener_base = ['Date', 'Year', 'Type', 'Country', 'Activity', 'Sex', 'Age']
# detectamos automáticamente qué columna del csv parece ser de fatalidad
fatal_cols_in_raw = [c for c in df.columns if re.search(r'f\s*a\s*t\s*a\s*l', c.lower())]
# combinamos sin duplicados manteniendo orden
mantener = list(dict.fromkeys(mantener_base + fatal_cols_in_raw))
# nos quedamos solo con esas columnas
df = df[[c for c in mantener if c in df.columns]].copy()
# =======================================================
# NO PROVOCADOS
# =======================================================
t = df['Type'].astype(str).str.lower().str.strip()
is_unprovoked = (
    t.str.contains('unprovok', na=False) |
    t.str.contains('unexpected', na=False) |
    t.str.contains('not provok', na=False) |
    (t.str.contains('non', na=False) & t.str.contains('provok', na=False))
)
df = df[is_unprovoked].copy()
df['Type'] = 'no provocado'
# =======================================================
# PAISES: NORMALIZAR + FILTRAR >= MIN_COUNT + CANÓNICO
# =======================================================
MIN_COUNT = 15  # umbral mínimo de casos por país
def norm_country(s):
    if pd.isna(s):
        return ""
    s = str(s).strip().lower()
    # quitar acentos
    s = ''.join(
        ch for ch in unicodedata.normalize('NFD', s)
        if unicodedata.category(ch) != 'Mn'
    )
    # quitar símbolos raros
    s = re.sub(r'[^\w\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    # normalizaciones específicas
    s = (
        s.replace("u s a", "usa")
         .replace("u s", "us")
         .replace("u k", "uk")
         .replace("u a e", "uae")
         .replace("r s a", "rsa")
    )
    return s
df["_ckey"] = df["Country"].apply(norm_country)
# contamos por país normalizado
key_counts = df['_ckey'].value_counts()
# nos quedamos con los países con >= MIN_COUNT casos
valid_keys = key_counts[key_counts >= MIN_COUNT].index
df = df[df['_ckey'].isin(valid_keys)].copy()
# elegimos forma canónica del país (la más frecuente escrita en los datos)
canon = df.groupby('_ckey')['Country'].agg(lambda s: s.value_counts().idxmax())
df['Country'] = df['_ckey'].map(canon)
df = df.drop(columns=['_ckey'])
# =======================================================
# SEXO Y EDAD
# =======================================================
# edad: sacar primer número, imputar mediana y castear a int
df['Age'] = (
    df['Age'].astype(str).str.extract(r'(\d+)')[0].astype(float)
)
df['Age'] = df['Age'].fillna(df['Age'].median()).astype(int)
# sexo: normalizar a M / F / Unknown
df['Sex'] = (
    df['Sex'].astype(str).str.upper().str.strip()
      .replace({'MALE': 'M', 'FEMALE': 'F'})
)
df['Sex'] = df['Sex'].where(df['Sex'].isin(['M', 'F']), 'Unknown')
# =======================================================
# FILTRAR SOLO AÑOS >= 2000 (usando Year)
# =======================================================
df['Year_num'] = pd.to_numeric(df['Year'], errors='coerce')
df = df[df['Year_num'] >= 2000].copy()
# =======================================================
# FECHAS (parseamos pero ya NO filtramos por esto)
# =======================================================
def clean_date(date_str, fallback_year=None):
    if pd.isna(date_str) or str(date_str).strip().lower() in ['nan', '']:
        return pd.NaT
    s = str(date_str)
    # quitar sufijos tipo '12th' -> '12'
    s = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', s, flags=re.IGNORECASE)
    # si falta año explícito en la fecha, añadimos el Year de esa fila
    if fallback_year and not re.search(r'\b\d{4}\b', s):
        s = f"{s} {int(fallback_year)}"
    # parseo flexible, dayfirst=True porque muchos registros van dd/mm
    return pd.to_datetime(s, dayfirst=True, errors='coerce')
yr = df['Year_num']
df['Date_parsed'] = [
    clean_date(d, y if not pd.isna(y) else None)
    for d, y in zip(df['Date'], yr)
]
# columnas derivadas de la fecha parseada
df['Month'] = df['Date_parsed'].dt.month
df['Year_from_Date'] = df['Date_parsed'].dt.year
# Nota: aquí NO eliminamos filas aunque Date_parsed sea NaT
# =======================================================
# FATALITY → "Fatal" / "No fatal" / "Desconocido"
# =======================================================
fatal_cols = [c for c in df.columns if re.search(r'f\s*a\s*t\s*a\s*l', c.lower())]
if fatal_cols:
    # renombramos la primera columna que matchee a 'Fatality'
    fatal_col = fatal_cols[0]
    df.rename(columns={fatal_col: 'Fatality'}, inplace=True)
    f = df['Fatality'].astype(str).str.strip().str.lower()
    def map_fatal(x: str) -> str:
        x = x.strip().lower()
        yes_set = {'y', 'yes', 'fatal', 'dead', 'deceased', 'sí', 'si'}
        no_set  = {'n', 'no', 'non fatal', 'non-fatal', 'nonfatal',
                   'alive', 'survived', 'not fatal', 'not-fatal'}
        unk_set = {'unknown', 'unk', 'n/a', 'na', '?', ''}
        if x in yes_set:
            return 'Fatal'
        if x in no_set:
            return 'No fatal'
        if x in unk_set:
            return 'Desconocido'
        # reglas de fallback si no coincide exactamente
        if ('not' in x or 'non' in x) and 'fatal' in x:
            return 'No fatal'
        if 'fatal' in x:  # solo si no hay 'not'/'non'
            return 'Fatal'
        return 'Desconocido'
    df['Fatality_clean'] = f.apply(map_fatal)
else:
    # si por lo que sea no había ninguna col con 'fatal' en el nombre
    df['Fatality_clean'] = 'Desconocido'
# =======================================================
# ACTIVITY
# =======================================================
df['Activity_clean'] = df['Activity'].astype(str).str.strip().str.title()
valid_activities = [
    'Swimming', 'Surfing', 'Spearfishing', 'Bathing', 'Snorkeling',
    'Bodyboarding', 'Diving', 'Wading', 'Standing', 'Fishing'
]
df['Activity_clean'] = df['Activity_clean'].apply(
    lambda x: x if x in valid_activities else 'Other'
)
# =======================================================
# CHECK OPCIONAL (puedes dejar esto para debug)
# =======================================================
print(f"Número de filas finales: {df.shape[0]}")
print(df[['Fatality', 'Fatality_clean']].head(20))


Número de filas finales: 2074
   Fatality Fatality_clean
0         N       No fatal
1         N       No fatal
2         N       No fatal
4         Y          Fatal
5         N       No fatal
6         N       No fatal
7         N       No fatal
8         N       No fatal
10        N       No fatal
12        N       No fatal
13        N       No fatal
14        N       No fatal
16        N       No fatal
17        N       No fatal
18        N       No fatal
20        N       No fatal
21        Y          Fatal
22        N       No fatal
23        N       No fatal
24        N       No fatal


In [10]:
total_ataques = len(df)
ataques_por_act = df['Activity_clean'].value_counts()
porcentaje_ataques = (ataques_por_act / total_ataques * 100).round(2)
print("Porcentaje de ataques por actividad (% sobre el total):")
print(porcentaje_ataques)
# calculamos por actividad:
# - total de ataques
# - ataques fatales
riesgo_df = (
    df
    .groupby('Activity_clean')
    .agg(
        ataques_totales = ('Activity_clean', 'size'),
        ataques_fatales = ('Fatality_clean', lambda x: (x == 'Fatal').sum())
    )
)
# añadimos % de mortalidad dentro de cada actividad
riesgo_df['riesgo_mortalidad_%'] = (
    riesgo_df['ataques_fatales'] / riesgo_df['ataques_totales'] * 100
).round(2)
print("Riesgo de mortalidad por actividad:")
print(riesgo_df)

Porcentaje de ataques por actividad (% sobre el total):
Activity_clean
Surfing         35.34
Other           29.22
Swimming        15.77
Spearfishing     5.88
Wading           4.24
Snorkeling       4.19
Standing         2.27
Fishing          1.54
Diving           1.40
Bathing          0.10
Bodyboarding     0.05
Name: count, dtype: float64
Riesgo de mortalidad por actividad:
                ataques_totales  ataques_fatales  riesgo_mortalidad_%
Activity_clean                                                       
Bathing                       2                0                 0.00
Bodyboarding                  1                0                 0.00
Diving                       29                6                20.69
Fishing                      32                3                 9.38
Other                       606               56                 9.24
Snorkeling                   87               12                13.79
Spearfishing                122               17               

In [11]:
total_ataques = len(df)
# 1. % de ataques por actividad (ordenado de más a menos)
porcentaje_ataques = (
    df['Activity_clean']
    .value_counts(normalize=True) * 100
)
porcentaje_ataques = porcentaje_ataques.round(2).sort_values(ascending=False)
print("Porcentaje de ataques por actividad (% sobre el total) [ordenado]:")
print(porcentaje_ataques)
# 2. Riesgo de mortalidad por actividad (ordenado de más a menos mortal)
riesgo_df = (
    df
    .groupby('Activity_clean')
    .agg(
        ataques_totales = ('Activity_clean', 'size'),
        ataques_fatales = ('Fatality_clean', lambda x: (x == 'Fatal').sum())
    )
)
riesgo_df['riesgo_mortalidad_%'] = (
    riesgo_df['ataques_fatales'] / riesgo_df['ataques_totales'] * 100
).round(2)
riesgo_df_sorted = riesgo_df.sort_values('riesgo_mortalidad_%', ascending=False)
print("\nRiesgo de mortalidad por actividad [ordenado]:")
print(riesgo_df_sorted)
# 3. Tabla resumen completa y ordenada por % de ataques
resumen = (
    (df['Activity_clean'].value_counts(normalize=True) * 100)
    .round(2)
    .rename('porcentaje_ataques_sobre_total_%')
    .to_frame()
).join(riesgo_df)
resumen['riesgo_mortalidad_%'] = (
    resumen['ataques_fatales'] / resumen['ataques_totales'] * 100
).round(2)
# ordenamos de más a menos común
resumen_sorted = resumen.sort_values('porcentaje_ataques_sobre_total_%', ascending=False)
print("\nResumen por actividad [ordenado por % de ataques]:")
print(resumen_sorted)

Porcentaje de ataques por actividad (% sobre el total) [ordenado]:
Activity_clean
Surfing         35.34
Other           29.22
Swimming        15.77
Spearfishing     5.88
Wading           4.24
Snorkeling       4.19
Standing         2.27
Fishing          1.54
Diving           1.40
Bathing          0.10
Bodyboarding     0.05
Name: proportion, dtype: float64

Riesgo de mortalidad por actividad [ordenado]:
                ataques_totales  ataques_fatales  riesgo_mortalidad_%
Activity_clean                                                       
Diving                       29                6                20.69
Spearfishing                122               17                13.93
Snorkeling                   87               12                13.79
Swimming                    327               44                13.46
Fishing                      32                3                 9.38
Other                       606               56                 9.24
Surfing                     733    